In [1]:
from Tools.scripts.nm2def import symbols
from dotenv import load_dotenv, dotenv_values
from binance.client import Client
import pandas as pd

In [2]:
config = dotenv_values(".env")
client = Client(api_key=config['BINANCE_TEST_KEY'], api_secret=config['BINANCE_TEST_SECRET'], testnet=True)

In [3]:
client

In [5]:
account = client.get_account()

In [8]:
df = pd.DataFrame(account['balances'])
df

,asset,free,locked
0,ETH,1.00000000,0.00000000
1,BTC,1.00000000,0.00000000
2,LTC,8.00000000,0.00000000
3,BNB,1.00000000,0.00000000
4,USDT,10000.00000000,0.00000000
...,...,...,...
388,LISTA,1487.00000000,0.00000000
389,ZRO,153.00000000,0.00000000
390,G,12602.00000000,0.00000000
391,BANANA,14.00000000,0.00000000


In [11]:
df.free = pd.to_numeric(df.free, errors='coerce')
df.locked = pd.to_numeric(df.locked, errors='coerce')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 393 entries, 0 to 392
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   asset   393 non-null    object 
 1   free    393 non-null    float64
 2   locked  393 non-null    float64
dtypes: float64(2), object(1)
memory usage: 9.3+ KB


In [12]:
client.get_asset_balance('BTC')

{'asset': 'BTC', 'free': '1.00000000', 'locked': '0.00000000'}

In [13]:
client.get_asset_balance('ETH')

{'asset': 'ETH', 'free': '1.00000000', 'locked': '0.00000000'}

In [21]:
timestamp = client._get_earliest_valid_timestamp(symbol='BTCUSDT', interval='1d')
timestamp

BinanceAPIException: APIError(code=-1121): Invalid symbol.

In [22]:
pd.to_datetime((timestamp), unit='ms')

Timestamp('2024-08-07 00:00:00')

In [17]:
def get_history(symbol, interval, start, end = None):
    bars = client.get_historical_klines(symbol=symbol, interval=interval, start_str=start, end_str=end, limit=1000)
    df = pd.DataFrame(bars)
    df['Date'] = pd.to_datetime(df.iloc[:,0], unit='ms')
    df.columns = [
        'Open Time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close Time', 'Quote Asset Volume', 'Number of Trades', "Taker Buy Base Asset Volume", 'Taker Buy Quote Asset Volume', 'Ignore', 'Date'
    ]
    df.set_index('Date', inplace=True)
    for column in df.columns:
        df[column] = pd.to_numeric(df[column], errors='coerce')
    return df

In [23]:
df = get_history('BTCUSDT', interval='1h', start=timestamp)
df

,Open Time,Open,High,Low,Close,Volume,Close Time,Quote Asset Volume,Number of Trades,Taker Buy Base Asset Volume,Taker Buy Quote Asset Volume,Ignore
Date,,,,,,,,,,,,
2024-08-07 13:00:00,1723035600000,57268.01,57552.44,21250.0,56825.99,17.78981,1723039199999,1.016193e+06,5437,10.83106,618778.235832,0
2024-08-07 14:00:00,1723039200000,56826.83,80000.00,21250.0,56119.84,18.77845,1723042799999,1.058372e+06,4648,11.95959,674118.946053,0
2024-08-07 15:00:00,1723042800000,56132.00,56350.75,21250.0,55908.27,19.53876,1723046399999,1.094926e+06,5071,12.95947,726204.567835,0
2024-08-07 16:00:00,1723046400000,55915.61,56290.04,21250.0,55886.47,18.96112,1723049999999,1.062444e+06,4865,11.35758,636582.658072,0
2024-08-07 17:00:00,1723050000000,55877.49,56214.84,21250.0,55628.28,17.64953,1723053599999,9.855920e+05,4538,12.57976,702470.655079,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-09-03 21:00:00,1725397200000,58215.96,59373.00,40106.0,58112.00,16.70378,1725400799999,9.648559e+05,3142,10.24549,595918.460213,0
2024-09-03 22:00:00,1725400800000,58115.11,58152.76,57864.0,57864.00,17.43289,1725404399999,1.011259e+06,3456,8.82154,511923.420349,0
2024-09-03 23:00:00,1725404400000,57864.99,57899.00,40073.0,57506.91,16.88427,1725407999999,9.583567e+05,3530,7.15259,412749.169103,0


In [26]:
from binance import AsyncClient, BinanceSocketManager

data = pd.DataFrame(columns = ["Open", "High", "Low", "Close", "Volume", "Complete"])

def stream_data(msg):
    global data
    event_time = pd.to_datetime(msg['E'], unit='ms')
    start_time = pd.to_datetime(msg['k']['t'], unit='ms')
    first = float(msg['k']['o'])
    high = float(msg['k']['h'])
    low = float(msg['k']['l'])
    close = float(msg['k']['c'])
    volume = float(msg['k']['v'])
    complete = float(msg['k']['x'])
    
    print(f"Time: {event_time} | Price {close}")
    
    data.loc[start_time] = [first, high, low, close, volume, complete]


async def main():
    client = await AsyncClient.create()
    bm = BinanceSocketManager(client)
    ts = bm.kline_socket(symbol='BTCUSDT', interval='1m')
    
    async with ts as tscm:
        for _ in range(50):
            res = await tscm.recv()
            stream_data(res)
    await client.close_connection()


In [27]:
await main()

Time: 2024-09-04 01:16:34.118000 | Price 56147.98
Time: 2024-09-04 01:16:36.156000 | Price 56149.99
Time: 2024-09-04 01:16:38.195000 | Price 56169.43
Time: 2024-09-04 01:16:40.575000 | Price 56160.69
Time: 2024-09-04 01:16:42.604000 | Price 56177.52
Time: 2024-09-04 01:16:44.671000 | Price 56174.71
Time: 2024-09-04 01:16:46.739000 | Price 56172.16
Time: 2024-09-04 01:16:48.814000 | Price 56190.0
Time: 2024-09-04 01:16:50.827000 | Price 56192.68
Time: 2024-09-04 01:16:52.983000 | Price 56195.76
Time: 2024-09-04 01:16:55.051000 | Price 56186.02
Time: 2024-09-04 01:16:57.067000 | Price 56191.54
Time: 2024-09-04 01:16:59.130000 | Price 56205.86
Time: 2024-09-04 01:17:00.006000 | Price 56211.1
Time: 2024-09-04 01:17:02.067000 | Price 56227.52
Time: 2024-09-04 01:17:04.075000 | Price 56261.09
Time: 2024-09-04 01:17:06.211000 | Price 56258.0
Time: 2024-09-04 01:17:08.287000 | Price 56235.99
Time: 2024-09-04 01:17:10.305000 | Price 56221.12
Time: 2024-09-04 01:17:12.422000 | Price 56191.11
Tim

In [28]:
account = client.get_account()
df = pd.DataFrame(account['balances'])
df

,asset,free,locked
0,ETH,1.00000000,0.00000000
1,BTC,1.00000000,0.00000000
2,LTC,8.00000000,0.00000000
3,BNB,1.00000000,0.00000000
4,USDT,10000.00000000,0.00000000
...,...,...,...
388,LISTA,1487.00000000,0.00000000
389,ZRO,153.00000000,0.00000000
390,G,12602.00000000,0.00000000
391,BANANA,14.00000000,0.00000000


In [29]:
order = client.create_order(symbol = 'BTCUSDT', side = 'BUY', type = 'MARKET', quantity = 0.1)
order

{'symbol': 'BTCUSDT',
 'orderId': 15425132,
 'orderListId': -1,
 'clientOrderId': 'Ea1YRkma6Y9bGPmWPZSol3',
 'transactTime': 1725412827600,
 'price': '0.00000000',
 'origQty': '0.10000000',
 'executedQty': '0.10000000',
 'cummulativeQuoteQty': '5617.99807230',
 'status': 'FILLED',
 'timeInForce': 'GTC',
 'type': 'MARKET',
 'side': 'BUY',
 'workingTime': 1725412827600,
 'fills': [{'price': '56112.34000000',
   'qty': '0.00202000',
   'commission': '0.00000000',
   'commissionAsset': 'BTC',
   'tradeId': 3301063},
  {'price': '56115.14000000',
   'qty': '0.00740000',
   'commission': '0.00000000',
   'commissionAsset': 'BTC',
   'tradeId': 3301064},
  {'price': '56115.89000000',
   'qty': '0.00312000',
   'commission': '0.00000000',
   'commissionAsset': 'BTC',
   'tradeId': 3301065},
  {'price': '56116.00000000',
   'qty': '0.00740000',
   'commission': '0.00000000',
   'commissionAsset': 'BTC',
   'tradeId': 3301066},
  {'price': '56130.00000000',
   'qty': '0.00891000',
   'commission

In [31]:
fills = pd.DataFrame(order['fills'])
fills

,price,qty,commission,commissionAsset,tradeId
0,56112.34000000,0.00202000,0.00000000,BTC,3301063
1,56115.14000000,0.00740000,0.00000000,BTC,3301064
2,56115.89000000,0.00312000,0.00000000,BTC,3301065
3,56116.00000000,0.00740000,0.00000000,BTC,3301066
4,56130.00000000,0.00891000,0.00000000,BTC,3301067
5,56130.33000000,0.00624000,0.00000000,BTC,3301068
6,56137.35000000,0.00714000,0.00000000,BTC,3301069
7,56146.88000000,0.00633000,0.00000000,BTC,3301070
8,56161.10000000,0.00481000,0.00000000,BTC,3301071
9,56161.11000000,0.00490000,0.00000000,BTC,3301072


In [33]:
fills.price = pd.to_numeric(fills.price, errors='coerce')
fills.price.mean()

np.float64(56338.63352941177)

In [36]:
num_cols = ['price', 'qty', 'commission']
for col in num_cols:
    fills[col] = pd.to_numeric(fills[col], errors='coerce')
fills.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   price            17 non-null     float64
 1   qty              17 non-null     float64
 2   commission       17 non-null     float64
 3   commissionAsset  17 non-null     object 
 4   tradeId          17 non-null     int64  
dtypes: float64(3), int64(1), object(1)
memory usage: 812.0+ bytes


In [37]:
fills.qty.sum()

np.float64(0.1)

In [38]:
fills.price.mul(fills.qty).sum()

np.float64(5617.9980723)

In [40]:
orderQuote = client.create_order(symbol = 'BTCUSDT', side = 'BUY', type = 'MARKET', quoteOrderQty = 1000)
orderQuote

{'symbol': 'BTCUSDT',
 'orderId': 15426774,
 'orderListId': -1,
 'clientOrderId': 'mfnhUvS5iX0o3SdhnpZsPy',
 'transactTime': 1725413521718,
 'price': '0.00000000',
 'origQty': '0.01776000',
 'executedQty': '0.01776000',
 'cummulativeQuoteQty': '999.44400000',
 'status': 'FILLED',
 'timeInForce': 'GTC',
 'type': 'MARKET',
 'side': 'BUY',
 'workingTime': 1725413521718,
 'fills': [{'price': '56275.00000000',
   'qty': '0.01776000',
   'commission': '0.00000000',
   'commissionAsset': 'BTC',
   'tradeId': 3301852}],
 'selfTradePreventionMode': 'EXPIRE_MAKER'}

In [43]:
avg_price = float(orderQuote['cummulativeQuoteQty']) / float(orderQuote['origQty'])
avg_price

56274.99999999999

In [76]:
account = client.get_account()
balances = pd.DataFrame(account['balances'])
balances

,asset,free,locked
0,ETH,1.00000000,0.00000000
1,BTC,1.01776000,0.00000000
2,LTC,8.00000000,0.00000000
3,BNB,1.00000000,0.00000000
4,USDT,9040.64997240,0.00000000
...,...,...,...
388,LISTA,1487.00000000,0.00000000
389,ZRO,153.00000000,0.00000000
390,G,12602.00000000,0.00000000
391,BANANA,14.00000000,0.00000000


In [75]:
sell_order = client.create_order(symbol = 'BTCUSDT', side = 'SELL', type = 'MARKET', quantity = 0.1)
sell_order

{'symbol': 'BTCUSDT',
 'orderId': 15441571,
 'orderListId': -1,
 'clientOrderId': 'WUm7no95QbxRdwvfUeArZF',
 'transactTime': 1725417864055,
 'price': '0.00000000',
 'origQty': '0.10000000',
 'executedQty': '0.10000000',
 'cummulativeQuoteQty': '5672.34911250',
 'status': 'FILLED',
 'timeInForce': 'GTC',
 'type': 'MARKET',
 'side': 'SELL',
 'workingTime': 1725417864055,
 'fills': [{'price': '56728.00000000',
   'qty': '0.00468000',
   'commission': '0.00000000',
   'commissionAsset': 'USDT',
   'tradeId': 3306133},
  {'price': '56727.77000000',
   'qty': '0.00697000',
   'commission': '0.00000000',
   'commissionAsset': 'USDT',
   'tradeId': 3306134},
  {'price': '56726.82000000',
   'qty': '0.00811000',
   'commission': '0.00000000',
   'commissionAsset': 'USDT',
   'tradeId': 3306135},
  {'price': '56726.76000000',
   'qty': '0.00459000',
   'commission': '0.00000000',
   'commissionAsset': 'USDT',
   'tradeId': 3306136},
  {'price': '56726.70000000',
   'qty': '0.00820000',
   'commi

In [46]:
account = client.get_account()
balances = pd.DataFrame(account['balances'])
balances

,asset,free,locked
0,ETH,1.00000000,0.00000000
1,BTC,1.01776000,0.00000000
2,LTC,8.00000000,0.00000000
3,BNB,1.00000000,0.00000000
4,USDT,9053.30085990,0.00000000
...,...,...,...
388,LISTA,1487.00000000,0.00000000
389,ZRO,153.00000000,0.00000000
390,G,12602.00000000,0.00000000
391,BANANA,14.00000000,0.00000000


In [62]:
client.get_symbol_ticker(symbol='BTCUSDT')

{'symbol': 'BTCUSDT', 'price': '56885.99000000'}

In [52]:
limitOrder = client.create_order(symbol = 'BTCUSDT', side = 'BUY', type = 'LIMIT', quantity = 0.1, timeInForce = "GTC", price = 56850)

In [57]:
limitOrder

{'symbol': 'BTCUSDT',
 'orderId': 15432247,
 'orderListId': -1,
 'clientOrderId': '5zqb2L67TijAOWmzx4eOAj',
 'transactTime': 1725415276877,
 'price': '56850.00000000',
 'origQty': '0.10000000',
 'executedQty': '0.00000000',
 'cummulativeQuoteQty': '0.00000000',
 'status': 'NEW',
 'timeInForce': 'GTC',
 'type': 'LIMIT',
 'side': 'BUY',
 'workingTime': 1725415276877,
 'fills': [],
 'selfTradePreventionMode': 'EXPIRE_MAKER'}

In [56]:
account = client.get_account()
balances = pd.DataFrame(account['balances'])
balances

,asset,free,locked
0,ETH,1.00000000,0.00000000
1,BTC,1.11776000,0.00000000
2,LTC,8.00000000,0.00000000
3,BNB,1.00000000,0.00000000
4,USDT,3368.30085990,0.00000000
...,...,...,...
388,LISTA,1487.00000000,0.00000000
389,ZRO,153.00000000,0.00000000
390,G,12602.00000000,0.00000000
391,BANANA,14.00000000,0.00000000


In [60]:
client.get_order(symbol = 'BTCUSDT', orderId = limitOrder['orderId'])

{'symbol': 'BTCUSDT',
 'orderId': 15432247,
 'orderListId': -1,
 'clientOrderId': '5zqb2L67TijAOWmzx4eOAj',
 'price': '56850.00000000',
 'origQty': '0.10000000',
 'executedQty': '0.10000000',
 'cummulativeQuoteQty': '5685.00000000',
 'status': 'FILLED',
 'timeInForce': 'GTC',
 'type': 'LIMIT',
 'side': 'BUY',
 'stopPrice': '0.00000000',
 'icebergQty': '0.00000000',
 'time': 1725415276877,
 'updateTime': 1725415295280,
 'isWorking': True,
 'workingTime': 1725415276877,
 'origQuoteOrderQty': '0.00000000',
 'selfTradePreventionMode': 'EXPIRE_MAKER'}

In [63]:
limitSellOrder = client.create_order(symbol = 'BTCUSDT', side = 'SELL', type = 'LIMIT', quantity = 0.1, timeInForce = "GTC", price = 56950)

In [64]:
limitSellOrder

{'symbol': 'BTCUSDT',
 'orderId': 15432957,
 'orderListId': -1,
 'clientOrderId': 'Wo9W6DVH88OTRgzsMHRSrk',
 'transactTime': 1725415485869,
 'price': '56950.00000000',
 'origQty': '0.10000000',
 'executedQty': '0.00000000',
 'cummulativeQuoteQty': '0.00000000',
 'status': 'NEW',
 'timeInForce': 'GTC',
 'type': 'LIMIT',
 'side': 'SELL',
 'workingTime': 1725415485869,
 'fills': [],
 'selfTradePreventionMode': 'EXPIRE_MAKER'}

In [65]:
client.get_order(symbol = 'BTCUSDT', orderId = limitSellOrder['orderId'])

{'symbol': 'BTCUSDT',
 'orderId': 15432957,
 'orderListId': -1,
 'clientOrderId': 'Wo9W6DVH88OTRgzsMHRSrk',
 'price': '56950.00000000',
 'origQty': '0.10000000',
 'executedQty': '0.00000000',
 'cummulativeQuoteQty': '0.00000000',
 'status': 'NEW',
 'timeInForce': 'GTC',
 'type': 'LIMIT',
 'side': 'SELL',
 'stopPrice': '0.00000000',
 'icebergQty': '0.00000000',
 'time': 1725415485869,
 'updateTime': 1725415485869,
 'isWorking': True,
 'workingTime': 1725415485869,
 'origQuoteOrderQty': '0.00000000',
 'selfTradePreventionMode': 'EXPIRE_MAKER'}

In [67]:
open_orders = client.get_open_orders(symbol = 'BTCUSDT')
open_orders

[{'symbol': 'BTCUSDT',
  'orderId': 15432957,
  'orderListId': -1,
  'clientOrderId': 'Wo9W6DVH88OTRgzsMHRSrk',
  'price': '56950.00000000',
  'origQty': '0.10000000',
  'executedQty': '0.00000000',
  'cummulativeQuoteQty': '0.00000000',
  'status': 'NEW',
  'timeInForce': 'GTC',
  'type': 'LIMIT',
  'side': 'SELL',
  'stopPrice': '0.00000000',
  'icebergQty': '0.00000000',
  'time': 1725415485869,
  'updateTime': 1725415485869,
  'isWorking': True,
  'workingTime': 1725415485869,
  'origQuoteOrderQty': '0.00000000',
  'selfTradePreventionMode': 'EXPIRE_MAKER'}]

In [68]:
pd.DataFrame(open_orders)

,symbol,orderId,orderListId,clientOrderId,price,origQty,executedQty,cummulativeQuoteQty,status,timeInForce,type,side,stopPrice,icebergQty,time,updateTime,isWorking,workingTime,origQuoteOrderQty,selfTradePreventionMode
0,BTCUSDT,15432957,-1,Wo9W6DVH88OTRgzsMHRSrk,56950.00000000,0.10000000,0.00000000,0.00000000,NEW,GTC,LIMIT,SELL,0.00000000,0.00000000,1725415485869,1725415485869,True,1725415485869,0.00000000,EXPIRE_MAKER


In [69]:
result = client.cancel_order(symbol = 'BTCUSDT', orderId = open_orders[0]['orderId'])

In [70]:
result

{'symbol': 'BTCUSDT',
 'origClientOrderId': 'Wo9W6DVH88OTRgzsMHRSrk',
 'orderId': 15432957,
 'orderListId': -1,
 'clientOrderId': 'WcVV2m8h7ROef9CeKyP5k5',
 'transactTime': 1725415577137,
 'price': '56950.00000000',
 'origQty': '0.10000000',
 'executedQty': '0.00000000',
 'cummulativeQuoteQty': '0.00000000',
 'status': 'CANCELED',
 'timeInForce': 'GTC',
 'type': 'LIMIT',
 'side': 'SELL',
 'selfTradePreventionMode': 'EXPIRE_MAKER'}

In [71]:
open_orders = client.get_open_orders(symbol = 'BTCUSDT')
open_orders

[]

In [74]:
all_orders = client.get_all_orders(symbol='BTCUSDT')
all_orders = pd.DataFrame(all_orders)
all_orders

,symbol,orderId,orderListId,clientOrderId,price,origQty,executedQty,cummulativeQuoteQty,status,timeInForce,type,side,stopPrice,icebergQty,time,updateTime,isWorking,workingTime,origQuoteOrderQty,selfTradePreventionMode
0,BTCUSDT,15425132,-1,Ea1YRkma6Y9bGPmWPZSol3,0.00000000,0.10000000,0.10000000,5617.99807230,FILLED,GTC,MARKET,BUY,0.00000000,0.00000000,1725412827600,1725412827600,True,1725412827600,0.00000000,EXPIRE_MAKER
1,BTCUSDT,15426774,-1,mfnhUvS5iX0o3SdhnpZsPy,0.00000000,0.01776000,0.01776000,999.44400000,FILLED,GTC,MARKET,BUY,0.00000000,0.00000000,1725413521718,1725413521718,True,1725413521718,1000.00000000,EXPIRE_MAKER
2,BTCUSDT,15429125,-1,79IeQDULzHWHejwCvmHMtb,0.00000000,0.10000000,0.10000000,5670.74293220,FILLED,GTC,MARKET,SELL,0.00000000,0.00000000,1725414375343,1725414375343,True,1725414375343,0.00000000,EXPIRE_MAKER
3,BTCUSDT,15432247,-1,5zqb2L67TijAOWmzx4eOAj,56850.00000000,0.10000000,0.10000000,5685.00000000,FILLED,GTC,LIMIT,BUY,0.00000000,0.00000000,1725415276877,1725415295280,True,1725415276877,0.00000000,EXPIRE_MAKER
4,BTCUSDT,15432957,-1,Wo9W6DVH88OTRgzsMHRSrk,56950.00000000,0.10000000,0.00000000,0.00000000,CANCELED,GTC,LIMIT,SELL,0.00000000,0.00000000,1725415485869,1725415577137,True,1725415485869,0.00000000,EXPIRE_MAKER


In [77]:
import asyncio
from binance import AsyncClient, BinanceSocketManager

stop_streaming = False

def simple_bot(msg):
    time = pd.to_datetime(msg['E'], unit = 'ms')
    price = float(msg['c'])
    print(f"Time: {time} | Price: {price}")
    if int(price) % 10 == 0:
        order = client.create_order(symbol='BTCUSDT', side='BUY', type='MARKET', quantity= 0.01)
        print("\n"+50*"-")
        print(f"Buy {order['executedQty']} for {order['cummulativeQuoteQty']} USDT")
        print(50 * "-" + "\n")
        global stop_streaming
        stop_streaming = True

In [79]:
async def main():
    client = await AsyncClient.create()
    bm = BinanceSocketManager(client)
    ts = bm.symbol_miniticker_socket(symbol='BTCUSDT')
    
    async with ts as tscm:
        while True:
            res = await tscm.recv()
            simple_bot(res)
            
            if stop_streaming:
                break
    await client.close_connection()

In [80]:
await main()

Time: 2024-09-04 02:47:48.813000 | Price: 56758.0
Time: 2024-09-04 02:47:49.761000 | Price: 56756.01
Time: 2024-09-04 02:47:50.433000 | Price: 56756.0
Time: 2024-09-04 02:47:51.656000 | Price: 56756.01
Time: 2024-09-04 02:47:52.240000 | Price: 56756.0
Time: 2024-09-04 02:47:53.801000 | Price: 56756.01
Time: 2024-09-04 02:47:54.869000 | Price: 56753.0
Time: 2024-09-04 02:47:56.202000 | Price: 56752.0
Time: 2024-09-04 02:47:56.902000 | Price: 56752.01
Time: 2024-09-04 02:47:58.166000 | Price: 56752.0
Time: 2024-09-04 02:47:58.454000 | Price: 56752.0
Time: 2024-09-04 02:48:00.157000 | Price: 56752.0
Time: 2024-09-04 02:48:01.075000 | Price: 56752.0
Time: 2024-09-04 02:48:02.204000 | Price: 56752.0
Time: 2024-09-04 02:48:02.576000 | Price: 56752.01
Time: 2024-09-04 02:48:04.205000 | Price: 56752.0
Time: 2024-09-04 02:48:05.211000 | Price: 56752.0
Time: 2024-09-04 02:48:05.848000 | Price: 56752.01
Time: 2024-09-04 02:48:07.164000 | Price: 56750.0

-------------------------------------------

In [81]:
account = client.get_account()
balances = pd.DataFrame(account['balances'])
balances

,asset,free,locked
0,ETH,1.00000000,0.00000000
1,BTC,1.03776000,0.00000000
2,LTC,8.00000000,0.00000000
3,BNB,1.00000000,0.00000000
4,USDT,7906.26908600,0.00000000
...,...,...,...
388,LISTA,1487.00000000,0.00000000
389,ZRO,153.00000000,0.00000000
390,G,12602.00000000,0.00000000
391,BANANA,14.00000000,0.00000000
